In [1]:
import os
import sys
sys.path.append(os.getcwd()+'/..')

from pdearena import utils
import torch
from pdearena.data.datamodule import PDEDataModule
from pdearena.lr_scheduler import LinearWarmupCosineAnnealingLR  # noqa: F401
from pdearena.models.pdemodel import PDEModel

from helper_functions.data import write_h5, read_h5_numpy
from helper_functions.general import touch_dir
from time import time
import numpy as np
import h5py
from tqdm import tqdm
import shutil
import _pickle as cPickle
import pickle 
import hdf5plugin


In [2]:
import sys
sys.path


['/mnt/c/Users/eid0112947/OneDrive - Eni/Documenti/RESEARCH/pdearena/pdedatagen',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python38.zip',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/lib-dynload',
 '',
 '/home/bonotto/workspace/miniconda3/envs/pdearena/lib/python3.8/site-packages',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/sci_ML/pdearena',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/ENI/Projects/Prove/helper_functions',
 '/mnt/c/Users/eid0112947/OneDrive - Eni/Documenti/RESEARCH/pdearena/pdedatagen/..',
 '/tmp/tmpr1i2au5j']

In [3]:
def hdf5_to_dict(file):
    d = dict()
    for key,val in file.items():
        if type(val) == h5py._hl.dataset.Dataset:
            d[key] = np.array(val)
        else:
            d[key] = hdf5_to_dict(val)
    return d

In [4]:
### Chosen the following saving strategies
'''
- hdf5plugin.Blosc(cname='lz4', clevel = 1)
- pickle
'''
compressor = hdf5plugin.Blosc(cname='lz4', clevel = 1)

input_folder = '../pdearena_data/NavierStokes-2D-32x32/'
target_folders = [
    '../pdearena_data/NavierStokes-2D-32x32_pkl/',
]
files = os.listdir(input_folder)


def convert_file(filein,fileout):
    with h5py.File(filein, 'r') as hf_source:
        d = hdf5_to_dict(hf_source)
        if 'pkl' in fileout:
            with open(fileout,"wb") as filehandler:
                cPickle.dump(d,filehandler,protocol=pickle.HIGHEST_PROTOCOL)
            # pass
        else:
            with h5py.File(fileout, "w") as h5f_target:
                mode = next(iter(d.keys()))
                dataset = h5f_target.create_group(mode)
                for k,v in d[mode].items():
                    # print('processing {}'.format(k))
                    dataset.create_dataset(
                        k, 
                        data = v,
                        shape = v.shape,
                        **compressor
                    )



for target_folder in target_folders:
    touch_dir(target_folder)
    
    for file in tqdm(files,total=len(files)):
        try:
            if 'yaml' not in file:
                fileout = file
                if 'pkl' in target_folder:
                    fileout = fileout[:-3] + '.pkl'
                convert_file(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
            else:
                fileout = file
                shutil.copyfile(
                    os.path.join(input_folder,file),
                    os.path.join(target_folder,fileout)
                )
        except:
            print(file,fileout)
        



100%|██████████| 312/312 [00:48<00:00,  6.46it/s]
